In [1]:
from omnidata.framework import get_builder
from omnilearn import util

No profile path provided (can be set with environment variable: FIG_PROFILE)
2022-09-01 16:41:43.543708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /is/ei/fleeb/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:
2022-09-01 16:41:43.543723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
opt = get_builder('optim')
opt

omnilearn.novo.optim.PytorchOptimizer

In [3]:
hparams = list(opt.plan('adam'))
hparams

[('lr', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be1c0>),
 ('beta1', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be220>),
 ('beta2', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be280>),
 ('eps', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be2e0>),
 ('weight_decay',
  <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be340>),
 ('ident', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be400>),
 ('amsgrad',
  <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be3a0>)]

In [4]:
lrp = hparams[0][1]

In [5]:
optim = opt.build()
optim

Adam ()

In [6]:
optim.lr

0.001

In [ ]:
list(optim.full_spec())

In [ ]:
from src import main

In [ ]:
list(main.Supervised_Model.full_spec())

In [ ]:
'asd'.split('.')

In [1]:
from omnibelt import nodes

In [2]:
Node = nodes.AutoTreeNode

In [3]:
g = Node.from_raw({'a': [12, 13], 'b': {'x': 1, 'y': 2}})

In [4]:
g.set('b.z', 100)

In [5]:
g.set('a.6', 20)
g.set('a.3', [])

In [6]:
g.payload

OrderedDict([('a',
              [12,
               13,
               omnibelt.nodes.LocalNode._empty_value,
               [],
               omnibelt.nodes.LocalNode._empty_value,
               omnibelt.nodes.LocalNode._empty_value,
               20]),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [7]:
f = dict(g.flatten())
f

{'a.0': 12, 'a.1': 13, 'a.3': [], 'a.6': 20, 'b.x': 1, 'b.y': 2, 'b.z': 100}

In [8]:
g2 = Node.from_raw(f)
g2.payload

OrderedDict([('a', OrderedDict([('0', 12), ('1', 13), ('3', []), ('6', 20)])),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [9]:
g2.convert_to_dense('a')
g2.payload

OrderedDict([('a',
              [12,
               13,
               omnibelt.nodes.LocalNode._empty_value,
               [],
               omnibelt.nodes.LocalNode._empty_value,
               omnibelt.nodes.LocalNode._empty_value,
               20]),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [11]:
g2.get('a.0').parent.payload

[12,
 13,
 omnibelt.nodes.LocalNode._empty_value,
 [],
 omnibelt.nodes.LocalNode._empty_value,
 omnibelt.nodes.LocalNode._empty_value,
 20]

In [22]:
from omnibelt import capturable_super, captured_super
from omnibelt.tricks import Capturable, captured

In [23]:
class A(Capturable):
    def captured_super_call(self, src, name, args, kwargs):
        print('!! captured super', self, src, name, args, kwargs)
        return super().captured_super_call(src, name, args, kwargs)

    def captured_method_call(self, src, fn, args, kwargs):
        print('!! captured method', self, src, fn, args, kwargs)
        return super().captured_method_call(src, fn, args, kwargs)

    def f(self, a=1):
        print('A.f', self, a)
        # print(super().f())

class B(A):
    x = -10

    def f(self, a=2):
        print('B.f', self, a)
        super().f()

class C(B):
    @captured
    def f(self, a=3):
        print('C.f', self, a)
        super().f()

class D(C):
    def f(self, a=4):
        print('D.f', self, a)
        super().f()

class E(D):
    @captured
    def f(self, a=5):
        print('E.f', self, a)
        super().f()

class F(E):
    def f(self, a=6):
        print('F.f', self, a)
        super().f()

class G(F):
    pass

In [24]:
A().f()
print()
B().f()
print()
C().f()
print()
D().f()
print()
E().f()
print()
F().f()
print()
G().f()

A.f <__main__.A object at 0x7f51a901b4c0> 1

B.f <__main__.B object at 0x7f51a901b4c0> 2
A.f <__main__.B object at 0x7f51a901b4c0> 1

!! captured method <__main__.C object at 0x7f51a901b4c0> <class '__main__.C'> <function C.f at 0x7f51a8fcca60> () {}
C.f <__main__.C object at 0x7f51a901b4c0> 3
!! captured super <__main__.C object at 0x7f51a901b4c0> <class '__main__.C'> f () {}
B.f <__main__.C object at 0x7f51a901b4c0> 2
A.f <__main__.C object at 0x7f51a901b4c0> 1

D.f <__main__.D object at 0x7f51a901b4c0> 4
!! captured method <__main__.D object at 0x7f51a901b4c0> <class '__main__.C'> <function C.f at 0x7f51a8fcca60> () {}
C.f <__main__.D object at 0x7f51a901b4c0> 3
!! captured super <__main__.D object at 0x7f51a901b4c0> <class '__main__.C'> f () {}
B.f <__main__.D object at 0x7f51a901b4c0> 2
A.f <__main__.D object at 0x7f51a901b4c0> 1

!! captured method <__main__.E object at 0x7f51a901b4c0> <class '__main__.E'> <function E.f at 0x7f51a8fcc820> () {}
E.f <__main__.E object at 0x7f51a90

In [ ]:
c.f.__closure__[0].cell_contents

In [ ]:
super(C, c).f

In [21]:
Foo.__init__

<future_method __init__ of Foo>

In [20]:
Foo()

AttributeError: 'NoneType' object has no attribute 'a'

In [18]:
Bar(100)

!! capturing <__main__.Bar object at 0x7f85a21f5400> <class '__main__.Bar'> __init__ () {}
1 2
1 2 100


In [6]:
Foo

__main__.Foo

In [378]:
class A:
    def __getattribute__(self, item):
        print(self, item)
        return super().__getattribute__(item)

class B(A):
    x = 10
    pass

class C(B):
    pass

In [379]:
c = C()

<__main__.C object at 0x7f1b55ed2c10> __class__
<__main__.C object at 0x7f1b55ed2c10> __class__
<__main__.C object at 0x7f1b55ed2c10> __class__
<__main__.C object at 0x7f1b55ed2c10> __class__


In [380]:
c.x

<__main__.C object at 0x7f1b58409310> x


10

In [ ]:
# class Beyond:
#     def f(self):
#         return
#     def f2(self):
#         return

class Dest(metaclass=MetaDest):
    pass
# MetaDest._type = Dest

class Shield(Dest):
    pass
# MetaShield._type = Shield

In [ ]:
s = Shield()
s

In [ ]:
s._skip_dest_flag

In [2]:
from omnibelt import method_wrapper

In [3]:
class customizable_super_type(type):
    # def __new__(cls, name, bases, attrs):
    #     return super().__new__(cls, name, bases, attrs)

    _super_hijacker_type = None
    _super_trigger_type = None
    _most_recent_origin_type = None

    # def __getattribute__(self, item):
    #     print('getattribute', self, item)
    #     return super().__getattribute__(item)

    def __getattribute__(self, item):
        hijacker = super().__getattribute__('_super_hijacker_type')
        trigger = super().__getattribute__('_super_trigger_type')
        origin = super().__getattribute__('_most_recent_origin_type')
        if self is hijacker and origin is not trigger:
            out = super().__getattribute__('_super_capture')(origin, item)
            # out = self._super_capture(origin, item)
        else:
            out = super().__getattribute__(item)
        # self._most_recent_origin_type = self
        setattr(super().__getattribute__('__class__'), '_most_recent_origin_type', self)
        print('updated origin', self, item)
        return out

        pass

    # def __getattr__(self, item):
    #     if self is self._super_hijacker_type and self._most_recent_origin_type is not self._super_trigger_type:
    #         out = self._super_capture(self._most_recent_origin_type, item)
    #     else:
    #         out = super().__getattribute__(item)
    #     self._most_recent_origin_type = self
    #     print('updated origin', self, item)
    #     return out

    class _super_capture:
        def __init__(self, origin, key):
            self.origin = origin
            self.key = key

        def __call__(self, obj, *args, **kwargs):
            print('running', self.origin, obj, self.key, args, kwargs)
            return getattr(super(self.origin, obj), self.key)(*args, **kwargs)


class _super_hijacker(metaclass=customizable_super_type): pass
class customizable_super(_super_hijacker):
    _super_hijacker_type = _super_hijacker
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)

        for key, value in cls.__dict__.items():
            if isinstance(value, hijack_super):
                value = value.fn
                if value.__closure__ is not None:
                    value.__closure__[0].cell_contents = cls._super_hijacker_type
                setattr(cls, key, value)
                print('updated', key, value.__closure__[0].cell_contents)

customizable_super_type._super_hijacker_type = _super_hijacker
customizable_super_type._super_trigger_type = customizable_super

class hijack_super(method_wrapper): pass

# def __get__(self, obj, cls=None):
#     if self.fn.__closure__ is not None:
#         self.fn.__closure__[0].cell_contents = _super_hijacker
#     return super().__get__(obj, cls=cls)



In [4]:
class A(customizable_super):
    def f(self, a=1):
        print(a)

class B(A):
    def f(self, a=2):
        print(a)

class C(B):
    # @hijack_super
    def f(self, a=3):
        print(a)
        super().f()

updated origin <class '__main__.A'> __dict__
updated origin <class '__main__.B'> __dict__
updated origin <class '__main__.C'> __dict__


In [5]:
C.f.__closure__[0].cell_contents = _super_hijacker

updated origin <class '__main__.C'> f


In [6]:
C.f

updated origin <class '__main__.C'> f


<function __main__.C.f(self, a=3)>

In [7]:
C.f.__closure__[0].cell_contents

updated origin <class '__main__.C'> f
updated origin <class '__main__._super_hijacker'> __class__


AttributeError: '_super_capture' object has no attribute '__name__'

updated origin <class '__main__._super_hijacker'> __class__


AttributeError: '_super_capture' object has no attribute '__name__'

updated origin <class '__main__._super_hijacker'> __class__


...

AttributeError: '_super_capture' object has no attribute '__name__'

updated origin <class '__main__._super_hijacker'> __class__


AttributeError: '_super_capture' object has no attribute '__name__'

updated origin <class '__main__._super_hijacker'> __class__


AttributeError: '_super_capture' object has no attribute '__name__'

In [67]:
c = C()

In [68]:
super(C, c).f

<bound method B.f of <__main__.C object at 0x000001EC893D5B50>>

In [52]:
c.f.__closure__[0].cell_contents

updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_
updated origin <class '__main__._super_hijacker'> _ipython_canary_method_should_not_exist_

__main__._super_hijacker

In [53]:
super(C, c).f

<bound method B.f of <__main__.C object at 0x000001EC89BBB790>>

In [54]:
c.f()

3


AttributeError: 'super' object has no attribute 'f'

In [5]:
# class Beyond:
#     def f(self):
#         return
#     def f2(self):
#         return

class Dest(metaclass=MetaDest):
    pass
# MetaDest._type = Dest

class Shield(Dest):
    pass
# MetaShield._type = Shield

In [6]:
s = Shield()
s

In [7]:
s._skip_dest_flag

True

In [160]:
class A(Shield):#, Beyond):
    def __init__(self, x=1):
        pass

    def f(self, a=1):
        print(a)
        # return a
class B(A):
    pass

class C(B):
    def __init__(self, x=2, y=3, **kwargs):
        super().__init__(x=x, **kwargs)

    def f(self, a=100):
        print(a)

class D(C):
    class E:
        def h(self, x=1):
            super()
            pass

    def __getattribute__(self, item):
        # print(item)
        return super().__getattribute__(item)

    def f2(self):
        pass

    # @super_to(C)
    def f(self, a=10):
        print(locals().keys())
        print(__class__)
        # print(globals())
        # print(test)
        # my_super = super
        # print(my_super().f)
        print(super)
        print(super())
        print(super().f)
        super().f()
        print(super(C, self).__thisclass__)
        print(super(C, self).__self_class__)
        print(super(C, self).__self__)
        return a

In [157]:
D.f

shield <class '__main__.D'> f
dest <class '__main__.D'> f


<function __main__.D.f(self, a=10)>

In [88]:
inspect.getattr_static(D, 'f').fn.__closure__[0].cell_contents# = C

In [83]:
D.f#.fn.__closure__

<bound method method_wrapper.apply_fn of <__main__.super_to object at 0x7f1b588534f0>>

In [69]:
sig = inspect.signature(D.f)
sig

<Signature (*args, **kwargs)>

In [70]:
def g(x=1, y=2):
    pass

In [103]:
d = D()
d.f()

dict_keys(['self', 'a', '__class__'])
<class '__main__.D'>
<bound method C.f of <__main__.D object at 0x7f1b57d79a30>>
<class 'super'>
<super: <class 'D'>, <D object>>
<bound method C.f of <__main__.D object at 0x7f1b57d79a30>>
100
<class '__main__.C'>
<class '__main__.D'>


10

In [47]:
fn = D.f
fn = D.f2
fn = g
fn = D.E.h
fns = [D.f, D().f, D.f2, D().f2, g, D.E.h]

In [51]:
cl[0][0].cell_contents = C

In [52]:
cl = [fn.__closure__ for fn in fns]
cl

[(<cell at 0x7f1b57cb8d90: type object at 0x55e47d4876b0>,),
 (<cell at 0x7f1b57cb8d90: type object at 0x55e47d4876b0>,),
 None,
 None,
 None,
 (<cell at 0x7f1b57cb8e20: type object at 0x55e47d506cd0>,)]

In [53]:
cl[0][0].cell_contents, cl[-1][0].cell_contents

(__main__.C, __main__.D.E)

In [11]:
c.cell_contents

__main__.D

In [54]:
D().f()

dict_keys(['self', 'a', '__class__'])
<class '__main__.C'>
<class 'super'>
<super: <class 'C'>, <D object>>
<bound method A.f of <__main__.D object at 0x7f1b57c3a4f0>>


10

In [4]:
sig.replace?

In [44]:
vs.nonlocals['test'] = 112

In [96]:
class my_super(super):
    def __init__(self, *args, **kwargs):
        print(args, kwargs)
        super().__init__(*args, **kwargs)

In [97]:
class X:
    def f(self,x=1):
        return x
class Y:
    def f(self,x=10):
        return my_super().f()
Y().f()

() {}


AttributeError: 'my_super' object has no attribute 'f'

In [41]:
d = D()

__class__
__class__
__class__
__class__


In [29]:
super()

RuntimeError: super(): no arguments

In [42]:
d.f()

f
dict_keys(['self', 'a', '__class__'])
<class '__main__.D'>


NameError: name 'test' is not defined

In [9]:
getattr(D, '__init__')

<function __main__.C.__init__(self, x=2, y=3, **kwargs)>

In [14]:
D.__dict__.get('__init__')

In [6]:
params = inspect.signature(D.__init__).parameters
params

mappingproxy({'self': <Parameter "self">,
              'x': <Parameter "x=2">,
              'y': <Parameter "y=3">,
              'kwargs': <Parameter "**kwargs">})

In [15]:
ps = list(params.values())

In [12]:
D.mro()

[__main__.D, __main__.C, __main__.B, __main__.A, object]

In [17]:
def f(*a):
    print(a)
def g(x):
    for i in range(x):
        yield i
f(*g(2))

(0, 1)


In [19]:
isinstance(A(), type)

False

In [ ]:
import inspect

In [ ]:
def f(x, y, *args, z=1):
    # print(x,y,z)
    return x,y,z

In [ ]:
ps = inspect.signature(f).parameters
ps

In [ ]:
for k,v in ps.items():
    print(k,v.kind)

In [ ]:
class prop:
    def __init__(self, x=None):
        print(x)
        self.x = x

    def __call__(self, y=None):
        print(y)
        self.y = y
        return self

    pass

In [ ]:
for k,v in ps.items():
    print(k,v.kind)

In [ ]:
class prop:
    def __init__(self, x=None):
        print(x)
        self.x = x

    def __call__(self, y=None):
        print(y)
        self.y = y
        return self

    pass

In [26]:
class prop:
    def __init__(self, x=None):
        print(x)
        self.x = x

    def __call__(self, y=None):
        print(y)
        self.y = y
        return self

    pass

In [36]:
def g():
    pass

class A:
    h = g

    @prop(h)
    def f(self):
        return 0

<function g at 0x00000251AFC949D0>
<function A.f at 0x00000251AFC94A60>


In [35]:
str(A.f.x.__qualname__)

'g'

In [28]:
A.f

In [29]:
dir(A.f.x)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [30]:
dir(A.f.y)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']